# **Intercomparison Demo**

**Author:** Adam Yang (ayang115@umd.edu) and Hannah Kerner (hkerner@asu.edu)

**Description** Demo for Intercomparison Script: extracts manually labeled test points from publicly-available land cover maps (Harvest, Copernicus, ESA, GLAD, etc) and evaluates performance metrics for each map.

In [1]:
# !earthengine authenticate

In [2]:
import ee
ee.Initialize()

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ee
from shapely import wkt

import sys
sys.path.append("..")
from src.compare_covermaps import *

In [4]:
maps = [v for v in TARGETS.values()]
countries = ["Togo", "Kenya", "Malawi", "Tanzania", "Mali", "Namibia", "Rwanda", "Uganda", "Zambia"]

In [5]:
covermap_test = TestCovermaps(test_countries=countries, covermaps=maps)

In [6]:
# Load the test data points
test_pts = covermap_test.get_test_points()

# Alternatively, read from existing file
# test_pts = pd.read_csv("../../intercomparison-data/extracted/test/tgo_ken_tza_mlw_test.csv").drop("Unnamed: 0", axis=1)
# test_pts["geometry"] = test_pts["geometry"].apply(wkt.loads)
# test_pts = gpd.GeoDataFrame(test_pts, crs="epsg:4326")

# Print a random sample of the dataframe
test_pts.sample(10)

/Users/hkerner/anaconda3/envs/landcover-mapping/lib/python3.8/site-packages/geopandas/array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)
/Users/hkerner/anaconda3/envs/landcover-mapping/lib/python3.8/site-packages/geopandas/array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)
/Users/hkerner/anaconda3/envs/landcover-mapping/lib/python3.8/site-packages/geopandas/array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)
/Users/hkerner/anaconda3/envs/landcover-mapping/l

,lat,lon,binary,country,subset,geometry
755,-11.042025,34.050666,0,Malawi,testing,POINT (34.05067 -11.04203)
3018,-2.180316,30.584732,1,Rwanda,testing,POINT (30.58473 -2.18032)
575,4.051988,40.791510,0,Kenya,testing,POINT (40.79151 4.05199)
1665,-5.464450,37.217278,0,Tanzania,testing,POINT (37.21728 -5.46445)
3203,2.203150,34.306901,0,Uganda,testing,POINT (34.30690 2.20315)
2290,16.659257,3.831315,0,Mali,testing,POINT (3.83131 16.65926)
2170,16.883836,0.911790,0,Mali,testing,POINT (0.91179 16.88384)
1084,-4.227690,35.730372,1,Tanzania,testing,POINT (35.73037 -4.22769)
2806,-18.327672,20.750271,0,Namibia,testing,POINT (20.75027 -18.32767)
2163,22.812717,-3.013848,0,Mali,testing,POINT (-3.01385 22.81272)


In [7]:
# Notes about warnings:
# Number of sampled points could be more if there are overlapping images in imagecollection
# could only be less if there was missing data in the map?
extracted = covermap_test.extract_covermaps(test_pts)

[Togo] sampling copernicus...
[Togo] sampling worldcover...
[Togo] sampling glad...
[Togo] sampling asap...
[Togo] sampling dynamicworld...
[Togo] sampling gfsad-gcep...
[Togo] sampling gfsad-lgrip...
[Togo] sampling digital-earth-africa...
[Togo] sampling esa-cci-africa...
[Togo] sampling globcover-v23...
[Togo] sampling globcover-v22...
[Togo] sampling harvest-crop-maps...
[Togo] sampling esri-lulc...
[Togo] sampling nabil-etal-2021...
[Kenya] sampling copernicus...
[Kenya] sampling worldcover...
[Kenya] sampling glad...
[Kenya] sampling asap...
[Kenya] sampling dynamicworld...
[Kenya] sampling gfsad-gcep...
[Kenya] sampling gfsad-lgrip...
[Kenya] sampling digital-earth-africa...
[Kenya] sampling esa-cci-africa...
[Kenya] sampling globcover-v23...
[Kenya] sampling globcover-v22...
[Kenya] sampling harvest-crop-maps...
[Kenya] sampling esri-lulc...
[Kenya] sampling nabil-etal-2021...
[Malawi] sampling copernicus...
[Malawi] sampling worldcover...
[Malawi] sampling glad...
[Malawi] sam

In [8]:
# Add ensemble classifications (mode = voting classifier)
comp_maps = ['copernicus', 'worldcover', 'glad', 'asap', 'dynamicworld', 'gfsad-gcep',
             'digital-earth-africa', 'esa-cci-africa', 'globcover-v23', 'esri-lulc', 'nabil-etal-2021']

for country in countries:
    ensemble_subset = extracted[country][comp_maps].mode(axis='columns')
    extracted[country]['ensemble-subset'] = ensemble_subset
    
covermap_test.sampled_maps = extracted
covermap_test.covermap_titles = covermap_test.covermap_titles + ['ensemble-subset']

In [9]:
_ = covermap_test.evaluate()
results = pd.concat(covermap_test.results)
results

evaluating maps...
dataset: copernicus | country: Togo
dataset: worldcover | country: Togo
dataset: glad | country: Togo
dataset: asap | country: Togo
dataset: dynamicworld | country: Togo
dataset: gfsad-gcep | country: Togo
dataset: gfsad-lgrip | country: Togo
dataset: digital-earth-africa | country: Togo
dataset: esa-cci-africa | country: Togo
dataset: globcover-v23 | country: Togo
dataset: globcover-v22 | country: Togo
dataset: harvest-crop-maps | country: Togo
dataset: esri-lulc | country: Togo
dataset: nabil-etal-2021 | country: Togo
dataset: ensemble-subset | country: Togo
dataset: copernicus | country: Kenya
dataset: worldcover | country: Kenya
dataset: glad | country: Kenya
dataset: asap | country: Kenya
dataset: dynamicworld | country: Kenya
dataset: gfsad-gcep | country: Kenya
dataset: gfsad-lgrip | country: Kenya
dataset: digital-earth-africa | country: Kenya
dataset: esa-cci-africa | country: Kenya
dataset: globcover-v23 | country: Kenya
dataset: globcover-v22 | country: Ke

/Users/hkerner/anaconda3/envs/landcover-mapping/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/hkerner/anaconda3/envs/landcover-mapping/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/hkerner/anaconda3/envs/landcover-mapping/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

dataset: ensemble-subset | country: Mali
dataset: copernicus | country: Namibia
dataset: worldcover | country: Namibia
dataset: glad | country: Namibia
dataset: asap | country: Namibia
dataset: dynamicworld | country: Namibia
dataset: gfsad-gcep | country: Namibia
dataset: gfsad-lgrip | country: Namibia
dataset: digital-earth-africa | country: Namibia
dataset: esa-cci-africa | country: Namibia
dataset: globcover-v23 | country: Namibia
dataset: globcover-v22 | country: Namibia
dataset: esri-lulc | country: Namibia
dataset: nabil-etal-2021 | country: Namibia
dataset: ensemble-subset | country: Namibia
dataset: copernicus | country: Rwanda
dataset: worldcover | country: Rwanda
dataset: glad | country: Rwanda
dataset: asap | country: Rwanda
dataset: dynamicworld | country: Rwanda
dataset: gfsad-gcep | country: Rwanda
dataset: gfsad-lgrip | country: Rwanda
dataset: digital-earth-africa | country: Rwanda
dataset: esa-cci-africa | country: Rwanda
dataset: globcover-v23 | country: Rwanda
datas

country  crop_f1  std_crop_f1  accuracy  std_acc  \
       dataset                                                            
Togo   copernicus         Togo     0.59         0.27      0.79     0.03   
       worldcover         Togo     0.70         0.33      0.86     0.03   
       glad               Togo     0.71         0.34      0.86     0.03   
       asap               Togo     0.45         0.15      0.64     0.03   
       dynamicworld       Togo     0.21         0.10      0.75     0.03   
...                        ...      ...          ...       ...      ...   
Zambia globcover-v23    Zambia     0.19         0.04      0.91     0.01   
       globcover-v22    Zambia     0.10         0.01      0.85     0.01   
       esri-lulc        Zambia     0.74         0.62      0.98     0.01   
       nabil-etal-2021  Zambia     0.56         0.37      0.95     0.01   
       ensemble-subset  Zambia     0.74         0.61      0.98     0.01   

                        crop_recall_pa  std_crop_pa  noncrop_recall_pa  \
       dataset                                                           
Togo   copernicus                 0.55         0.06               0.88   
       worldcover                 0.57         0.05               0.98   
       glad                       0.59         0.05               0.97   
       asap                       0.53         0.06               0.68   
       dynamicworld               0.12         0.02               0.99   
...                                ...          ...                ...   
Zambia globcover-v23              0.25         0.09               0.94   
       globcover-v22              0.20         0.09               0.88   
       esri-lulc                  0.70         0.09               0.99   
       nabil-etal-2021            0.75         0.09               0.96   
       ensemble-subset            0.80         0.08               0.99   

                        std_noncrop_pa  crop_precision_ua  std_crop_ua  \
       dataset                                                           
Togo   copernicus                 0.02               0.64         0.07   
       worldcover                 0.01               0.91         0.05   
       glad                       0.01               0.88         0.06   
       asap                       0.02               0.39         0.06   
       dynamicworld               0.01               0.86         0.14   
...                                ...                ...          ...   
Zambia globcover-v23              0.00               0.15         0.06   
       globcover-v22              0.00               0.07         0.03   
       esri-lulc                  0.00               0.78         0.10   
       nabil-etal-2021            0.01               0.44         0.09   
       ensemble-subset            0.00               0.70         0.10   

                        noncrop_precision_ua  std_noncrop_ua  crop_support  \
       dataset                                                               
Togo   copernicus                       0.83            0.03            51   
       worldcover                       0.85            0.03            51   
       glad                             0.86            0.03            51   
       asap                             0.79            0.04            51   
       dynamicworld                     0.74            0.03            51   
...                                      ...             ...           ...   
Zambia globcover-v23                    0.97            0.01            20   
       globcover-v22                    0.96            0.01            20   
       esri-lulc                        0.99            0.01            20   
       nabil-etal-2021                  0.99            0.00            20   
       ensemble-subset                  0.99            0.00            20   

                        noncrop_support   tn  fp  fn  tp  
       dataset                                 

In [10]:
results.to_csv("../notebooks/intercomparison-results.csv")